In [9]:
%load_ext autoreload
%autoreload 2

import os

from dataset import SportsDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
sports = "soccer"

if sports == "soccer":
    metrica_files = ["match1.csv", "match2.csv", "match3_valid.csv"]
    metrica_paths = [f"data/metrica_traces/{f}" for f in metrica_files]
    data_paths = metrica_paths[:-1]

elif sports == "basketball":
    nba_files = os.listdir("data/nba_traces")
    nba_paths = [f"data/nba_traces/{f}" for f in nba_files]
    nba_paths.sort()
    data_paths = nba_paths[:10]

else:  # sports == "afootball"
    nfl_paths = ["data/nfl_traces/nfl_train.csv", "data/nfl_traces/nfl_test.csv"]
    data_paths = nfl_paths[:-1]

In [34]:
train_dataset = SportsDataset(
    sports=sports,
    data_paths=data_paths,
    n_features=6,
    normalize=True,
    flip_pitch=True
)
len(train_dataset)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:07<00:00,  3.83s/it]


9098

In [35]:
train_dataset[0][0].shape, train_dataset[0][1].shape

(torch.Size([200, 132]), (200, 2))